# Using the Cochran-Mantel-Haenszel (CMH) statistic to learn about masculinity in America

FiveThirtyEight, a web site about polling, sports, and statistics, worked with a few other parties to run a survey of men's relation to masculinity in the #meToo era. The discussion is here:
https://fivethirtyeight.com/features/what-do-men-think-it-means-to-be-a-man/
If you haven't read the article yet, or have questions about the data collection, you are encouraged to have a look now.

The analysis is noticeably univariate: it mostly discusses each question responses in turn, without many cross-tabulations like "How does the likelihood of self-perceived masculinity differ between married and single?" This is not to fault the authors, who are writing a short article for generalists.

But the staff at FiveThirtyEight have graciously and openly posted the data. So we can do further analysis including the above question, or whether men are more likely to pay on dates when they self-report as high-masculinity or when they want to be _perceived_ as masculine, or whether being heterosexual raises the chance of self-reporting as high-masculinity.

This page supplements <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3197362">An Analysis of U.S. Domestic Migration via Subset-stable Measures of Administrative Data</a>, a paper analyzing 82 million moves made by members of the U.S. formal economy, 2001-2015.

To properly do such an undertaking, the paper develops the Cochran-Mantel-Haenszel (CMH) statistic to answer questions about the relationship between factors. For example,

Q: What is the likelihood of moving for married relative to singles?  
A: Marrieds are less likely to move than singles.  
Q: What if we control for having kids and a mortgage?  
A: Marrieds are much more likely to move than singles.

This page will:

* Ask lots of intersting questions about the relationship between masculinity and other opinions and factors.
* Introduce you to the CMH statistic and the Python CMH package.

Let's get on with the analysis.

##  First, download the data and clean it a little.

The next three cells provide the functions to do so, then run them to produce a Pandas data frame named `d`. This is standard data prep.

Many of your questions about the survey questions may be answered by the full instrument itself, at https://github.com/fivethirtyeight/data/blob/master/masculinity-survey/masculinity-survey.pdf.

In [1]:
from urllib.request import urlopen
import pandas as pd  
                                            
Data_URL = "https://github.com/fivethirtyeight/data/blob/master/masculinity-survey/raw-responses.csv?raw=true" 
 
def get_data():    
    """Download a copy of the survey if we don't already have it. Return a data frame with the observations."""
    try:
        return pd.read_csv(open("survey.csv", 'rb'))
    except FileNotFoundError:                     
        in_csv = urlopen(Data_URL).read().decode('utf-8')
        f = open("survey.csv", 'w')                      
        for data in in_csv:                              
            f.write(data)
        return pd.read_csv(open("survey.csv", 'rb'))

In [2]:
def prep_data(d):
    """The columns have generic names; give them something more readable.
       Then recode some of the codes to numeric values."""
    d.rename(columns= {
        'q0001': 'masc_self_rate',        
        'q0002': 'perception_importance', 
        'q0005': 'societal_pressure',
        'q0010_0007': 'advantage_at_work',
        'q0011_0004': 'disadvantage_at_work',
        'q0018': 'pays_on_dates',         
        'q0021': 'concerns_about_too_far',
        'q0024': 'married',
        'q0025': 'kids',  
        'q0026': 'sexual_orientation',  
        'q0027': 'age',   
        'q0028': 'race',  
        }, inplace = True)

    """Encode text values to numeric. For control variables this will not actually be necessary."""
    likert_λ = (lambda x:
        1 if (x.startswith('Somewhat') or x.startswith('Very')) else 
        0 if (x.startswith('Not very') or x.startswith('Not at all'))
        else -1)
    none_λ = lambda x: 0 if str(x).startswith("None") else 1
    
    """Encode the masculinity self-rating, and listwise delete those that did not reply to this key question."""
    d.loc[:,"masc_self_rate"] = d["masc_self_rate"].apply(likert_λ)
    d.loc[:, "perception_importance"] = d["perception_importance"].apply(likert_λ)
    d = d.loc[d["masc_self_rate"]>=0]
    d = d.loc[d["perception_importance"]>=0]

    d.loc[:, "advantage_at_work"] = d["advantage_at_work"].apply(none_λ)
    d.loc[:, "disadvantage_at_work"] = d["disadvantage_at_work"].apply(none_λ)
    d.loc[:, "sexual_orientation"] = d["sexual_orientation"].apply(lambda x: 0 if x == "Straight" else 1)
    d.loc[:, "married"] = d["married"].apply(lambda x: 1 if x == "Married" else 0)
    d.loc[:, "age3"] = d["age3"].apply(lambda x: 0 if x == "18 - 34" else 1 if x == "35 - 64" else 3)
    return d

In [3]:
d = prep_data(get_data())

## Now for some risk ratios

Now that we have the data set, we can start asking about how variables relate.

For example, how does being married relate to the chance of self-describing as "somewhat" or "very" masculine? The first way to answer this is a simple crosstab, as shown below. It is a survey with weighted results, so it is preferable to aggregate (`agg`) using the sum of weights, not simple observation counts.

As per the encodings above, unmarried=0, married=1, self-report as high masculinity=1, not as 0.

In [4]:
d.groupby(["married","masc_self_rate"]).agg({"weight": sum})

weight
married masc_self_rate            
0       0               127.765599
        1               525.307171
1       0                43.740838
        1               450.619175

Among the unmarried, the likelihood or _risk_ of self-reporting as somewhat/very masculine is (count of married who self-report as hi-masc) divided by (count of marrieds), which the table shows to be about 695/(695+177).
Similarly for the married. These are the two ratios to compare.

It is worth writing this down in general notation to make clear what the CMH statistic is doing.

The table of possibilities, including married=no and high masculinity self-rate=yes, married=yes and high masculinity-self-rate=no, ... looks like this:

|               |single     |  married  | 
|---------------|-----------:|-----------:|
|  high  | h$_y$ m$_n$ | h$_y$ m$_y$ |
|  low  | h$_n$ m$_n$ | h$_n$ m$_y$ |

The risk of high self-rate given single is the chance of singles reporting as high masculinity over the chance that somebody is single: $$ \frac{h_y m_n}{(h_y m_n + h_n m_n)}$$

Similiarly for the chance that a married person self-rates as high masculinity. Then the risk ratio is indeed the ratio of the two risks. The numerator is the same as the denominator, but all $m_y$s instead of the $m_n$s in the denominator:

$$ \frac{\frac{h_y m_y}{(h_y m_y + h_n m_y)}}{\frac{h_y m_n}{(h_y m_n + h_n m_n)}}$$

The denominator can be flipped to reduce this fraction-of-fractions to a more legible (and more useful) simple fraction:
$$ \frac{(h_y m_y)(h_y m_n + h_n m_n)}{(h_y m_n)(h_y m_y + h_n m_y)}$$


The CMH calculation with no controls gives exactly that risk ratio:


In [5]:
from cmh import cmh
mar_to_masc = cmh(d, "married", "masc_self_rate", "weight", [])
print(f"The risk of self-reporting as somehwat/very masculine for marrieds relative to singles: {mar_to_masc}")

The risk of self-reporting as somehwat/very masculine for marrieds relative to singles: 1.1332209122528545


So, the married are about 13% more likely to self-report as somewhat/very masculine.

The order of arguments to the `cmh` function are "independent" followed by "dependent"; picture an arrow `married → masc_self_rate`. Is it a measure of causality? That is for you to decide, because like all statistical measures, the CMH statistic advises but does not prove causality.

We can ask the CMH calculator to be a little more verbose in how it did the math. It will present a table giving the total weight for dependent=yes and independent=yes (`dyiy`), then dependent=yes and independent=no (`dyin`), and so on. These numbers will match the crosstab above. The numerator and denominator of the un-compounded fraction above is also given.

In [6]:
print(f'The calculated CMH: {cmh(d, "married", "masc_self_rate", "weight", [], verbose=True)}')

Independent: married, dependent: masc_self_rate, controls: []
         dyiy        dyin       dniy        dnin       weight      num  \
1                                                                        
1  450.619175  525.307171  43.740838  127.765599  1147.432783  0.22352   

        den  
1            
1  0.197243  
The calculated CMH: 1.1332209122528545


## Like the real world, the CMH is asymmetric

The masculine → married and married → masculine questions are distinct and not necessarily numerically related. Indeed, the chance of being married given a self-report of higher masculinity, relative to those with a self-report of lower masculinity, is noticeably larger than the likelihood of self-reporting higher masculinity given being married, relative to single, at 81% versus 12%:

In [7]:
print(f"""
masculinity self-rate → married
{cmh(d, "masc_self_rate", "married", "weight", [])}
""")


masculinity self-rate → married
1.8104475755260079



To drive home the point about how asymmetric the CMH statistic can be, let's ask these two questions about self-reported masculinity and the second question "How important is it to you that others see you as masculine?":

perception → masculinity:
How does the chance of giving high importance to being perceived as masculine rise for those with high masculinity self-reports relative to others?


and masculinity → perception: 
How does the chance of self-rating as somewhat/very masculine rise for those who report high importance to being perceived as masculine relative to others?

In [8]:
print(f"""
perception → masculinity self-rate
{cmh(d, "perception_importance", "masc_self_rate", "weight", [])}

masculinity self-rate → perception
{cmh(d, "masc_self_rate", "perception_importance", "weight", [])}
""")


perception → masculinity self-rate
1.3901236173090672

masculinity self-rate → perception
1.9286459946184804



So: men who care whether others perceive them as masculine are 39% more likely to self-report as masculine relative to people who don't perceive it as important.

But men who self-report as high-masculinity are on the way to twice as likely to think that everybody else cares about this than people who don't self-report as high-masculinity.

## Controlling for confounding factors

But there are all sorts of other issues. Maybe this is about having children. Married people are more likely to have kids, and maybe fathers feel more masculine.

Above, the controls were empty (the last argument to the calls to `cmh` were `[]`), and the married → high-masculinity self-rate CMH was 113%. Let's control for kids. Now, the CMH calculation generates a row in its grouped table for those with kids and those without, and finds the elements of the risk ratio for both sets. Then the aggregate risk ratio is the sum of numerators over the sum of denominators.

In [9]:
print(cmh(d, "married", "masc_self_rate", "weight", ["kids"], verbose=True))

Independent: married, dependent: masc_self_rate, controls: ['kids']
                    dyiy        dyin       dniy        dnin      weight  \
kids                                                                      
Has children  356.378401  201.500139  22.272502   23.115162  603.266204   
No children    93.523366  322.633410  21.468335  103.605786  541.230897   

                   num       den  
kids                              
Has children  0.115642  0.110224  
No children   0.064189  0.059740  
1.058050642996197


Indeed, the change in risk of self-reporting as somewhat/very masculine given married versus non-married is smaller—kids really did have an effect.

It seems natural to reverse the story and ask how having kids affects self-report of masculinity, controlling for marriage.

To do this, we need to account for a technical detail: above, we converted many columns of the data set to numbers, but did not do so with the "has children" column. The CMH function allows you to provide a function to evaluate whether the value of some column an observation has or does not have the property you are asking questions about. Here, the function is simple enough, `1 if kids=="Has children" else 0`, that we could have just done the text-to-number conversion above. But the facility is general enough that it can be used for less obvious situations.

Back to the question at hand, controlling for marriage, the risk of self-reporting as masculine rises 15% as we shift from not having kids to having them:

In [10]:
print(f"""kids → high-masculinity self-rate, controlling for marital status:
{cmh(d, "kids", "masc_self_rate", "weight", ["married"],
              indep_c=(lambda kids: 1 if kids=="Has children" else 0))}
     
high-masculinity self-rate → kids, controlling for marital status:
{cmh(d, "masc_self_rate", "kids", "weight", ["married"],
              dep_c=(lambda kids: 1 if kids=="Has children" else 0))}
""")

kids → high-masculinity self-rate, controlling for marital status:
1.1746561745178334
     
high-masculinity self-rate → kids, controlling for marital status:
1.8242351616259085



This is a survey of attitudes in the United States, so we have to take race into account. We can add race to the list of controls, so our controls are now `["race", "kids"]`.

Controlling for race has a minimal effect, but by producing a verbose aggregation table, we see why: the weighted survey is heavily White. The White-to-Asian ratio in the general U.S. population is about ten-to-one, and here is 23-to-one; similarly for other minorities. There are a lot of choices to be made in selecting survey weights, especially in a situation where only one weights column can be provided, and the designers chose weights that focus on other aspects, perhaps age or sexual orientation.

In [11]:
print(f"""
Controlling for race and kids:
{cmh(d, "married", "masc_self_rate", "weight", ["race", "kids"], verbose=True)}

Controlling only for kids:
{cmh(d, "married", "masc_self_rate", "weight", ["kids"])}
""")

Independent: married, dependent: masc_self_rate, controls: ['kids', 'race']
                             dyiy        dyin       dniy       dnin  \
kids         race                                                     
Has children Asian       6.498116    4.043315   0.000000   0.000000   
             Black      35.701493   34.346141   1.047586   8.643674   
             Hispanic   56.601465   39.839144   0.495502   0.215332   
             Other      14.644500   15.771335   1.708185   0.097685   
             White     242.932826  107.500204  19.021230  14.158471   
No children  Asian       0.248532   21.255771   1.889128   3.811718   
             Black       3.045584   49.421789   0.000000  21.180150   
             Hispanic   24.232309   43.663806   3.509484  15.419392   
             Other       3.066757   13.747157   0.102593   1.027961   
             White      62.930184  194.544887  15.967130  62.166563   

                           weight       num       den  
kids         ra

In fact, if we do the same check only on rows of the data set where `d["race"]=="White"`, almost nothing changes. Let's compare only Whites, everybody, then only non-Whites.

In [12]:
print(cmh(d.loc[d["race"]=="White"], "married", "masc_self_rate", "weight", ["kids"]))
print(cmh(d, "married", "masc_self_rate", "weight", ["kids"]))
print(cmh(d.loc[d["race"]!="White"], "married", "masc_self_rate", "weight", ["kids"]))

1.0506701948808865
1.058050642996197
1.0824741544054528


/usr/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Here is one last useful fact about the CMH statistic, a novel result proven in the migration paper linked at the top of this analysis, referred to there as _subset stability_.

We broke the full population into two parts, White and non-White, and got two CMH statistics, and the aggregate CMH statistic was in between. That is, if I start with the married → high-masculinity self-rate CMH statistic, and then I tell you that the person is White, then you will adjust your expectation of the effect of married → masc downward for that case. If I tell you that the person is non-White, then you will adjust your expectations upward.

This is extremely natural, and the above paper proves that the CMH statistic is the _only_ aggregate risk ratio among those admissible that guarantees this property. Without it, we would have the awkward situation where you could start with a baseline, and then I give you more information about the observation, and then you adjust upward no matter what I told you.

That concludes the methodological part of this essay: the CMH statistic is an excellent option for asking questions about how variables relate, controlling for others. The controls are literal, comparing only those married with kids to others married with kids, and subset statistics have the stability property demonstrated by Whites/everybody/non-Whites. An appendix to the above paper lists several other manners in which the CMH statistic improves over correlation-based statistics like regression coefficients.

Other issues, such as bootstrapping confidence intervals or using the lambdas to handle categorical data are omitted due to space considerations. There is also far more detail in the appendix of the above-linked paper.

In terms of masculinity and how it relates to other factors, let's knock out a few more statistics.

Are people who mark themselves as high-masculinity more likely to be heterosexual?

In [13]:
print(f"""
non-heterosexual → importance of perception self-rate:
{cmh(d, "sexual_orientation", "perception_importance", "weight", ["married", "kids"])}
     
importance of perception → non-heterosexual 
{cmh(d, "perception_importance", "sexual_orientation", "weight", ["married", "kids"])}
""")


non-heterosexual → importance of perception self-rate:
0.8916424966482716
     
importance of perception → non-heterosexual 
0.7571845351822163



Are men more likely to pay on a date when they self-report as masculine, or when they feel that being perceived as masculine is important?

In [14]:
pay_λ=lambda x: 1 if x=="Always" or x=="Often" else 0

print(f"""
perception → Often/always pays on dates
{cmh(d, "perception_importance", "pays_on_dates", "weight", [], dep_c=pay_λ)}

masculinity self-rate → Often/always pays on dates
{cmh(d, "masc_self_rate", "pays_on_dates", "weight", [], dep_c=pay_λ)}

Now repeat, with controls:

perception → Often/always pays on dates, controlling for masculinity self-report
{cmh(d, "perception_importance", "pays_on_dates", "weight", ["masc_self_rate"], dep_c=pay_λ)}

masculinity self-rate → Often/always pays on dates, controlling for importance of perception
{cmh(d, "masc_self_rate", "pays_on_dates", "weight", ["perception_importance"], dep_c=pay_λ)}
""")


perception → Often/always pays on dates
1.4101662122101937

masculinity self-rate → Often/always pays on dates
1.5304566727171374

Now repeat, with controls:

perception → Often/always pays on dates, controlling for masculinity self-report
1.2934765621776398

masculinity self-rate → Often/always pays on dates, controlling for importance of perception
1.365996146981868



The survey asked respondents whether being male had any advantages or disadvantages at work. Several options were given, but a large percentage marked _None_ for both questions. How do the likelihood of marking some non-none value change for high-masculinity self-raters versus others?

In [15]:
print(f"""
masculinity self-rate → Being male has advantages at work
{cmh(d, "masc_self_rate", "advantage_at_work", "weight", [])}

masculinity self-rate → Being male has disadvantages at work
{cmh(d, "masc_self_rate", "disadvantage_at_work", "weight", [])}
""")


masculinity self-rate → Being male has advantages at work
0.9143449984720041

masculinity self-rate → Being male has disadvantages at work
1.121007080521328



PS for those of you not familiar with Jupyter: this is a living document. If you have cmh.py installed, you can modify the code to ask more questions, or compare the results to your favorite alternative measures of controlled relationships.